In [53]:
!pip install pyarrow

In [54]:
!python -V

Python 3.9.12


In [55]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer

In [56]:

df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [57]:
num_columns = df.shape[1]
print("Number of columns:", num_columns)

Number of columns: 19


In [58]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])



In [59]:

df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60


std_duration = df['duration'].std()
print("Standard deviation of trip durations:", std_duration)

Standard deviation of trip durations: 42.594351241920904


In [60]:
df2 = df

In [61]:

initial_record_count = len(df)




In [62]:
filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
filtered_record_count = len(filtered_df)

In [63]:
fraction_left = filtered_record_count / initial_record_count
print("Fraction of records left after removing outliers:", fraction_left)

Fraction of records left after removing outliers: 0.9812202822125979


In [64]:
df = df[['PULocationID', 'DOLocationID']]

df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)


/tmp/ipykernel_41972/3633769171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PULocationID'] = df['PULocationID'].astype(str)
/tmp/ipykernel_41972/3633769171.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOLocationID'] = df['DOLocationID'].astype(str)


In [65]:
data_dicts = df.to_dict(orient='records')

In [66]:
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(data_dicts)

In [67]:
X_train.shape[1]


518

In [68]:

y_train = df2.duration.values

In [69]:

len(dv.feature_names_)

518

In [70]:

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [74]:

y_pred = lr.predict(X_train)

In [75]:

mean_squared_error(y_train, y_pred, squared=False)

41.99649230697585

In [ ]:
df_test = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")